In [1]:
from deeppavlov import build_model
model = build_model('punctuation_large.json')

[nltk_data] Downloading package punkt to /home/kravtsova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kravtsova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kravtsova/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kravtsova/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


2020-06-03 01:11:41.759 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /cephfs/home/kravtsova/hse/thesis/punctuation/punct_plain_large.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-06-03 01:12:04.312 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /cephfs/home/kravtsova/hse/thesis/punctuation/model_plain_large]



INFO:tensorflow:Restoring parameters from /cephfs/home/kravtsova/hse/thesis/punctuation/model_plain_large


In [2]:
#old fucked up
with open('RULEC-GEC_test_original_ya_corrected.txt', 'r', encoding='utf8') as f:
    data = f.readlines()
data = [x.strip('\n') for x in data]

In [121]:
with open('RULEC-GEC_test_ya_speller.txt', 'r', encoding='utf8') as f:
    data = f.readlines()
data = [x.strip() for x in data]

In [2]:
with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/excluded_test_ya_speller.txt', 'r', encoding='utf8') as f:
    data = f.readlines()
data = [x.strip() for x in data]

In [104]:
def normalize(text):
    return ' '.join(word.strip(punct) for word in text.split())
data_wo_punct = [normalize(x) for x in data]

In [105]:
with open('RULEC-GEC_test_ya_speller_wo_punct.txt', 'w', encoding='utf8') as f:
    for sent in data_wo_punct:
        f.write(sent + '\n')

In [ ]:
with open('RULEC-GEC_test_ya_speller_wo_punct.txt', 'r', encoding='utf8') as f:
    data = f.readlines()
data = [x.strip() for x in data]

In [2]:
with open('RozovskayaRothTACL2018-dataset/RULEC-GEC_test_original.txt', 'r', encoding='utf8') as f:
    data = f.readlines()
data = [x.strip() for x in data]

In [3]:
tokens_batch, punct_batch = model.batched_call(data, batch_size=16)

In [4]:
punct = '!.?:;—,…-«»`'

In [5]:
#new
updated = []
for tokens, punctuation in zip(tokens_batch, punct_batch):
    sent = []
    for i, (token, p) in enumerate(zip(tokens, punctuation)):
        sent.append(token)
        try:
            if (p is not None) and (p != tokens[i]) and (p != tokens[i+1]) and (
                tokens[i+2] not in punct) and (
                tokens[i+1] not in punct):
                sent.append(p)
        except IndexError:
             continue
    updated.append(sent)

In [6]:
wo_none = [' '.join(x for x in sent if x != 'NONE') for sent in updated]

In [97]:
tokens_batch[4385]

['Потом',
 'Он',
 'призвал',
 'к',
 'Лазаря',
 'и',
 'сказал',
 ',',
 'иди',
 'вон',
 '!',
 'Лазарь',
 'появился',
 ',',
 'живой',
 ',',
 'завернул',
 'в',
 'полотне',
 '.']

In [1]:
punct_batch[4385]

NameError: name 'punct_batch' is not defined

In [119]:
updated[3632]

['После',
 'NONE',
 'восемнадцатой',
 'NONE',
 'поправки',
 'NONE',
 'к',
 'NONE',
 'конституции',
 'США',
 '«',
 'NONE',
 'сухой',
 'закон',
 '»',
 'NONE',
 'в',
 '1919',
 ',',
 'NONE',
 'многие',
 'NONE',
 'пивоварения',
 'NONE',
 'закрылись',
 'NONE',
 'или',
 'NONE',
 'шли',
 'NONE',
 'на',
 'NONE',
 'чёрный',
 'рынок',
 '.']

In [111]:
wo_none[4385]

'Потом Он призвал к Лазаря и сказал : иди вон , Лазарь появился живой , завернул в полотне'

In [112]:
data[12]

'Но стоит отметить  что он использует просторечный пример  молодежный журнал  чтобы пояснить свою идею'

In [7]:
with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/excluded_test_ya_speller_punct.txt', 'w', encoding='utf8') as f:
    for sent in wo_none:
        f.write(sent + '\n')

#### Yandex Speller

In [8]:
with open('RULEC-GEC_test_original_punct.txt', 'r') as f:
    test = f.readlines()
test = [x.strip() for x in test]

In [9]:
import requests
def ya_correction(text):
    url = 'https://speller.yandex.net/services/spellservice.json/checkText'
    r = requests.post(url, data=f'text={text}'.encode(), headers={'Content-Type': 'application/x-www-form-urlencoded'})
    corrected = text
    for c in reversed(r.json()):
        if c.get('s'):
            corrected = corrected[:c['pos']] + c['s'][0] + corrected[c['pos'] + c['len']:] 
    return corrected

In [10]:
with open('RULEC-GEC_test_original_punct_ya_speller.txt', 'w', encoding='utf8') as f:
    for sent in test:
        f.write(ya_correction(sent) + '\n')